Задача: <br> Для каждого знака препинания из тестовой выборки предсказать, является ли он концом предложения. 

In [1]:
import pandas as pd
import numpy as np
import json
import io
import re
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
import sys 
sys.path

sys.path.append("/usr/local/lib/python2.7/dist-packages/")
#'/usr/local/lib/python2.7/dist-packages'
print sys.path
import xgboost as xgb

['', '/home/fox/anaconda2/lib/python27.zip', '/home/fox/anaconda2/lib/python2.7', '/home/fox/anaconda2/lib/python2.7/plat-linux2', '/home/fox/anaconda2/lib/python2.7/lib-tk', '/home/fox/anaconda2/lib/python2.7/lib-old', '/home/fox/anaconda2/lib/python2.7/lib-dynload', '/home/fox/.local/lib/python2.7/site-packages', '/home/fox/anaconda2/lib/python2.7/site-packages', '/home/fox/gym', '/home/fox/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/home/fox/.ipython', '/usr/local/lib/python2.7/dist-packages/']


/home/fox/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_data = []
with io.open('train_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        train_data.append(d)

In [4]:
test_data = []
with io.open('test_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        test_data.append(d)

In [5]:
end_of_sentences = set()
for paragraph in train_data:
    for sentence in paragraph['Sentences']:
        end_of_sentences.add(sentence[-1])

In [6]:
for symbol in end_of_sentences:
    print(symbol)

!
"
…
.
»
?


In [7]:
end_of_sentences_list = []
for paragraph in train_data:
    for sentence in paragraph['Sentences']:
        end_of_sentences_list += [sentence[-1]]

In [8]:
counter = Counter(end_of_sentences_list)
for i in counter:
    print i, counter[i]


! 1747
" 69
… 931
. 61167
» 313
? 2591


In [9]:
good_ends = [u'!', u'…', u'.', u'?']

## train_data приводим в форму test_data

In [10]:
k = 0
Index = 0
train_reform = []
for k in range(len(train_data)):
    pos = - 1
    true_positions = []
    for sent in train_data[k][u'Sentences']:
        pos += len(sent) 
        true_positions += [pos]
        pos += 1

    string = train_data[k]['Paragraph']
    Marks = []
    positions = []
    pos = 0
    for i in re.findall(re.compile(u'[!"…\.»?]', re.U), string):
        pos = string.find(i, pos + 1, len(string))
        positions += [pos]
    for pos in positions:
        Index += 1
        if pos in true_positions:
            label = True
        else:
            label = False
        Marks += [{u'Index': Index, u'Pos': pos, u'Mark': string[pos], u'label': label}]
    train_reform += [{'Paragraph': string, 'Marks': Marks}]
    

In [11]:
train_reform[:1]

[{'Marks': [{u'Index': 1, u'Mark': u'\xbb', u'Pos': 29, u'label': False},
   {u'Index': 2, u'Mark': u'.', u'Pos': 84, u'label': True},
   {u'Index': 3, u'Mark': u'\xbb', u'Pos': 135, u'label': False},
   {u'Index': 4, u'Mark': u'.', u'Pos': 136, u'label': True},
   {u'Index': 5, u'Mark': u'\u2026', u'Pos': 193, u'label': False},
   {u'Index': 6, u'Mark': u'\xbb', u'Pos': 194, u'label': False},
   {u'Index': 7, u'Mark': u'\xbb', u'Pos': 215, u'label': False},
   {u'Index': 8, u'Mark': u'\xbb', u'Pos': 281, u'label': False},
   {u'Index': 9, u'Mark': u'\xbb', u'Pos': 329, u'label': False},
   {u'Index': 10, u'Mark': u'.', u'Pos': 375, u'label': True}],
  'Paragraph': u'\u0412\u0435\u043b\u0438\u043a\u043e\u043b\u0435\u043f\u043d\u0430\u044f \xab\u0428\u043a\u043e\u043b\u0430 \u0437\u043b\u043e\u0441\u043b\u043e\u0432\u0438\u044f\xbb \u0432\u0435\u0440\u043d\u0443\u043b\u0430\u0441\u044c \u0432 \u044d\u0444\u0438\u0440 \u043f\u043e\u0441\u043b\u0435 \u043b\u0435\u0442\u043d\u0438\u0445 \u

## Extraxt features

In [12]:
def prepare_to_onehot(data):
    X = []
    k = 0
    for i in data:
        k += 1
        for Mark in i['Marks']:
            x = []
            for char in i['Paragraph'][Mark['Pos'] - 4: Mark['Pos'] + 6]:
                x += [ord(char)]
            
            if len(x) < 10:
                if len(x):
                    for j in range(10 - len(x)):
                        x += [0]
                else:
                    x = [0,0,0,0,0,0,0,0,0,0]
            X += [x]
    return X

In [13]:
enc = OneHotEncoder()
def extract_features(data):
    X = []
    for j in data:
        
        for i in j['Marks']:
            x = []
            ############### 0
            if i['Mark'] in good_ends:
                x += [1]
            else:
                x += [0]
            '''
            ############### 0.1
            if len(j['Paragraph']) > i['Pos'] + 1:
                #print j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 3]
                if re.findall(re.compile(u'[а-я|ё]\. [А-Я|Ё][^\.]', re.U), 
                j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            '''
            ############### 0.1
            if len(j['Paragraph']) - 1 == i['Pos']:
                x += [1]
            else:
                x += [0]
            ############### 0.2
            if len(j['Paragraph']) > i['Pos'] + 1:
                #print j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 3]
                if re.findall(re.compile(u'\.\.\. ', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            ############### 0.3
            if len(j['Paragraph']) > i['Pos'] + 1:
                #print j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 3]
                if re.findall(re.compile(u'[а-я|]\)\. [А-Я|Ё][^\.]', re.U), 
                              j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            ############### 1
            if len(j['Paragraph']) > i['Pos'] + 1:
                #print j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 3]
                if re.findall(re.compile(u'[А-Я|Ё]\. [А-Я|Ё][^\.]', re.U), 
                              j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            ################ 2
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'[\.!?"»…] [А-Я|Ё][^\.]', re.U), 
                              j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            ################ 3
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'[А-Я|Ё][а-я|ё]\. [А-Я|Ё][^\.]', re.U), 
                              j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 4
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'[А-Я|Ё]\.[А-Я|Ё]\. [А-Я|Ё][^\.]', re.U), 
                              j['Paragraph'][i['Pos'] - 5 : i['Pos'] + 6]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            
            ################ 5
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u' г\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 6
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'гг\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 7
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'т\.д\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 8
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'т\.к\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 9
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'т\.п\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 10
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'т\.е\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 11
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'н\.\э.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 12
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'вв\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 13
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u' в\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 14
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u' им\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 15
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u' м\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            ################ 16
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u' ул\.', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [0]
            
            ################ 17
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'\.»', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            ################ 18
            if len(j['Paragraph']) > i['Pos'] + 1:
                if re.findall(re.compile(u'…»', re.U), j['Paragraph'][i['Pos'] - 3 : i['Pos'] + 4]):
                    x += [1]
                else:
                    x += [0]
            else:
                x += [1]
            
            ################ 19
            if len(j['Paragraph']) < 10:
                x += [1]    
            else:
                x += [0]
            
            
            X += [x]
            
    X = np.array(X)
    enc.fit(prepare_to_onehot(train_reform) + prepare_to_onehot(test_data))
    
    X_2 = prepare_to_onehot(data)
    
    X_enc = enc.transform(X_2).toarray()
    
    X_new = []
    for i in range(len(X)):
        X_new  += [np.append(X[i], X_enc[i])] 
    X_new = np.array(X_new)
    X_enc = X = 0
    return X_new
     
    #return X

In [14]:
y = []
for i in train_reform:
    for Mark in i['Marks']:
        y += [Mark['label']]

In [15]:
y = np.array(y)

In [16]:
X = np.array(extract_features(train_reform))

## Local

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [18]:
cl = xgb.XGBClassifier(n_estimators=200, base_score=0.5, reg_lambda = 1) 

In [19]:
cl.fit(X_train, y_train) 

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [20]:
y_predict = cl.predict(X_test)

In [21]:
from sklearn.metrics import roc_auc_score

In [22]:
roc_auc_score(y_predict, y_test)

0.98754919842617972

## To Kaggle

In [23]:
X_train, y_train = X, y

In [24]:
X_test = np.array(extract_features(test_data))

In [25]:
cl = xgb.XGBClassifier(n_estimators=200, base_score=0.5, reg_lambda = 1) 

In [26]:
cl.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [27]:
y_test = cl.predict(X_test)

In [28]:
result = map(lambda x: 1 if x == True else 0, y_test)

In [29]:
print Counter(result)
len(result)

Counter({1: 17997, 0: 8479})


26476

In [30]:
df = pd.DataFrame(result, columns=['Mark'], index=range(1,26477))
df.index.name = 'Id'

In [32]:
df.to_csv("OneHot_XGB_GoodEnds_Submission.csv")